In [3]:
import tensorflow as tf
from transformers import BertTokenizer
import sys
sys.path.append('/kaggle/input/bertmodel/bert_model')
from bert import minBert
from bert import EncoderLayer, SelfAttention, FeedForward, BaseAttention
from positional_embedding import PositionalEmbedding
sys.path.append('/kaggle/input/bertmodel/contrastive_learning')
from contrastive_learning import SupervisedContrastiveModel

In [4]:
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
min_bert_layer = tf.keras.models.load_model(
    '/kaggle/input/bertmodel/min_bert_layer_after_training_qqp_triplet_v5.keras',
    custom_objects={
        "minBert": minBert,
        "EncoderLayer": EncoderLayer,
        "SelfAttention": SelfAttention,
        "FeedForward": FeedForward,
        "BaseAttention" : BaseAttention,
        "PositionalEmbedding" : PositionalEmbedding
    }
)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/ke

In [8]:
!pip install sentence_transformers

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.8 MB/s eta 0:00:0000:01


In [9]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
from datasets import load_dataset

# Load SNLI dataset
dataset = load_dataset("embedding-data/QQP_triplets")

README.md:   0%|          | 0.00/6.27k [00:00<?, ?B/s]

quora_duplicate_triplets.jsonl:   0%|          | 0.00/183M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/101762 [00:00<?, ? examples/s]

In [11]:
def prepare_data(dataset):
    anchors = []
    positives = []
    negatives = []

    sample_list = []

    for sample in dataset['set']:
        if len(sample['neg']) != 30: continue
        if len(anchors) == 1000: break
        anchors.append(sample['query'])
        sample_list.append(sample['pos'] + sample['neg'])

    # Tạo tf.data.Dataset
    dataset = {
        'anchor_sentences': anchors,
        'sample_sentences': sample_list
    }

    return dataset

In [12]:
test_dataset = prepare_data(dataset['train'][70000: 72000])

In [13]:
queries = test_dataset['anchor_sentences']
sample_sentences_list = test_dataset['sample_sentences']


In [14]:
def find_most_similar_for_SimCSE_Bert(query, dataset, model, tokenizer):
    query_encoding = tokenizer(query, return_tensors="tf", add_special_tokens=True, padding='max_length', truncation=True, max_length=256)
    query_embedding = model(query_encoding["input_ids"], training=False)[:,0,:]
    dataset_embeddings = model(tokenizer(dataset, return_tensors="tf", add_special_tokens=True, padding='max_length', truncation=True, max_length=256)["input_ids"], training=False)[:,0,:]
    similarities = tf.keras.losses.cosine_similarity(query_embedding, dataset_embeddings)
    return tf.argmax(-similarities).numpy()


In [15]:
def find_most_similar_for_SentenceTransformer(query, dataset, model):
    query_embedding = model.encode(query)
    dataset_embeddings = model.encode(dataset)
    similarities = tf.keras.losses.cosine_similarity(query_embedding, dataset_embeddings)
    return tf.argmax(-similarities).numpy()


In [16]:
correct_count = 0
for i in range(len(queries)):
    most_similar = find_most_similar_for_SimCSE_Bert(queries[i], sample_sentences_list[i], min_bert_layer, tokenizer)
    if most_similar == 0:
        correct_count += 1
print(f"SimCSE_Bert: correct: {correct_count} sentences, percentage: {correct_count / len(queries)}")

/opt/conda/lib/python3.10/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/ke

SimCSE_Bert: correct: 823 sentences, percentage: 0.823


In [ ]:
correct_count = 0
for i in range(len(queries)):
    most_similar = find_most_similar_for_SentenceTransformer(queries[i], sample_sentences_list[i], model)
    if most_similar == 0:
        correct_count += 1

In [18]:
print(f"SentenceTransformer: correct: {correct_count} sentences, percentage: {correct_count / len(queries)}")

SentenceTransformer: correct: 944 sentences, percentage: 0.944
